# OOR Evaluation

In [ ]:
import os

from objectherkenning_openbare_ruimte.performance_evaluation_pipeline.source.oor_evaluation import (
    OOREvaluation, tba_result_to_df, per_image_result_to_df, custom_coco_result_to_df
)

In [ ]:
# Size of the images (width, height)
img_shape = (1280, 720)
gt_base_dir = "../datasets/oor/processed-merged-batches-first-official-training-dataset-oor"
pred_base_dir = "../datasets/oor/inference/processed_merged_v2"
output_dir = ""

models = ["yolov8m_1280_oor_v2_best", "yolov8_1280_oor_v2_noble_sweep_15", "yolov8_1280_oor_v2_cerulean_sweep_25"]

splits = ["train", "val", "test"]

## Run full evaluation

In [ ]:
model_name = models[2]
pred_model_dir = os.path.join(pred_base_dir, model_name)

OOREval = OOREvaluation(
    ground_truth_base_folder=gt_base_dir,
    predictions_base_folder=pred_model_dir,
    output_folder=output_dir,
    predictions_image_shape=img_shape,
    model_name=model_name
)

In [ ]:
# Total Blurred Area evaluation
tba_results = OOREval.tba_evaluation()
tba_df = tba_result_to_df(tba_results)
tba_df.head()

In [ ]:
# Per Image evaluation
per_image_results = OOREval.per_image_evaluation()
per_image_df = per_image_result_to_df(per_image_results)
per_image_df.head()

In [ ]:
# Custom COCO evaluation
coco_results = OOREval.coco_evaluation()
coco_df = custom_coco_result_to_df(coco_results)
coco_df.head()

## Demo

In [ ]:
# For demo

import pandas as pd

pd.set_option('display.float_format', '{:.2f}'.format)

cols_to_show = [
    "Model",
    "Split",
    "Object Class",
    "AP@50_all",
    "AP@50_small",
    "AP@50_medium",
    "AP@50_large",
    "AR@50_all",
    "AR@50_small",
    "AR@50_medium",
    "AR@50_large",
]

# demo_df = coco_df[(coco_df["model_size"]=="m") & (coco_df["img_size"].isin((1024, 1920)))]
demo_df = coco_df
demo_df = demo_df[cols_to_show]

In [ ]:
demo_df[demo_df["Object Class"]=="all"]

In [ ]:
demo_df[demo_df["Object Class"]=="container"]

In [ ]:
demo_df[demo_df["Object Class"]=="person"]

## Detailed look into one specific model / run

In [ ]:
split = splits[1]
model = models[2]

gt_annotations_folder = f"{gt_base_dir}/labels/{split}"

In [ ]:
# Load frame metadata

import pathlib
import geopandas as gpd
from typing import List, Union

from cvtoolkit.datasets.yolo_labels_dataset import YoloLabelsDataset

metadata_folder = "../../datasets/oor/metadata"

RD_CRS = "EPSG:28992"  # CRS code for the Dutch Rijksdriehoek coordinate system
LAT_LON_CRS = "EPSG:4326"  # CRS code for WGS84 latitude/longitude coordinate system

def metadata_to_video_name(metadata_name: str) -> str:
    metadata_split = metadata_name.split(sep="-", maxsplit=1)
    return f"{metadata_split[0]}-0-{metadata_split[1]}"

def load_metadata_csv(metadata_file: str) -> pd.DataFrame:
    df = pd.read_csv(metadata_file)
    video_name = metadata_to_video_name(pathlib.Path(metadata_file).stem)
    df["frame_name"] = [f"{video_name}_{frame_id:04}" for frame_id in df["new_frame_id"]]
    return df.set_index("frame_name")

def get_target_cls_file_names(yolo_annotations_folder: str, target_cls: Union[ObjectClass, None] = None) -> List[str]:
    yolo_dataset = YoloLabelsDataset(
        folder_path=yolo_annotations_folder,
        image_area=img_shape[0]*img_shape[1],
    )
    if target_cls:
        yolo_dataset.filter_by_class(target_cls.value)
    target_labels = yolo_dataset._filtered_labels
    return [k for k, v in target_labels.items() if len(v) > 0]

metadata_files = pathlib.Path(metadata_folder).glob("*.csv")
metadata_df = pd.concat(
    [load_metadata_csv(metadata_file) for metadata_file in metadata_files]
)

metadata_gdf = gpd.GeoDataFrame(
    metadata_df,
    geometry=gpd.points_from_xy(
        x=metadata_df.gps_lon,
        y=metadata_df.gps_lat,
        crs=LAT_LON_CRS,
    ),
).to_crs(RD_CRS)

del metadata_df, metadata_files

In [ ]:
# Get all detections of containers

# Ground truth
gt_container_names = get_target_cls_file_names(gt_annotations_folder, ObjectClass.container)
keep_index = [frame in gt_container_names for frame in metadata_gdf.index]
gt_gdf = metadata_gdf[keep_index]
gt_gdf = gt_gdf[["gps_state", "geometry"]]

# Predictions
pred_folder = f"{pred_base_dir}/{model}/labels/{split}"

pred_container_names = get_target_cls_file_names(pred_folder, ObjectClass.container)
keep_index = [frame in pred_container_names for frame in metadata_gdf.index]
pred_gdf = metadata_gdf[keep_index]
pred_gdf = pred_gdf[["gps_state", "geometry"]]

In [ ]:
# Compute distances between ground truth and detections
gt_gdf["distance"] = gt_gdf["geometry"].distance(pred_gdf["geometry"].unary_union)
pred_gdf["distance"] = pred_gdf["geometry"].distance(gt_gdf["geometry"].unary_union)

In [ ]:
# Compute distance statistics
import numpy as np

stats = {
    "distance": np.arange(0, 26, 5),
    "fnr": [],
    "fpr": [],
}

gt_total = len(gt_gdf)
pred_total = len(pred_gdf)

for dst in stats["distance"]:
    fn = np.count_nonzero(gt_gdf["distance"] > dst)
    fp = np.count_nonzero(pred_gdf["distance"] > dst)
    stats["fnr"].append(fn/gt_total)
    stats["fpr"].append(fp/pred_total)

stats

In [ ]:
# Plot results on a map

import matplotlib.pyplot as plt

joined_gdf = gt_gdf.join(pred_gdf, how="outer", lsuffix="_gt", rsuffix="_pred")

f, ax = plt.subplots()
joined_gdf.set_geometry("geometry_gt").plot(ax=ax, markersize=20)
joined_gdf.set_geometry("geometry_pred").plot(ax=ax, color="red", markersize=5)

plt.savefig("val_map.png")
plt.show()

## Show stored CSV results

In [ ]:
import os
import pandas as pd

In [ ]:
results_dir = "../datasets/oor/evaluation"
model_name = "yolov8m_1280_v2.1"

per_image_classes = ("container", "mobile_toilet", "scaffolding")
coco_classes = ("person", "license_plate", "container")

tba_file = os.path.join(results_dir, model_name, f"{model_name}-tba-eval.csv")
per_image_file = os.path.join(results_dir, model_name, f"{model_name}-per-image-eval.csv")
coco_file = os.path.join(results_dir, model_name, f"{model_name}-custom-coco-eval.csv")

tba_results = pd.read_csv(tba_file, index_col=0)
per_image_results = pd.read_csv(per_image_file, index_col=0)
coco_results = pd.read_csv(coco_file, index_col=0)

per_image_results = per_image_results[per_image_results["Object Class"].isin(per_image_classes)]
coco_results = coco_results[coco_results["Object Class"].isin(coco_classes)]

In [ ]:
tba_results

In [ ]:
per_image_results[(per_image_results["Size"]=="all")].sort_values(by="Object Class")

In [ ]:
per_image_results[(per_image_results["Object Class"]=="container")]

In [ ]:
coco_results.sort_values(by="Object Class")